# Proposal 3

$$
\mathcal{L} = \mathcal{L}_c + \lambda_e \cdot || \bm{p^*_A} \odot \bm{(\theta - \theta^*_A) }||_1 + \lambda_f \cdot || \bm{(1 - p)} \odot \bm{\theta} ||_1
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUNV1
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/proposal_3.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'lambda_e': 10.0,
    'lambda_f': 10.0,
    'eps': 0.0001
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [10]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MWUNV1(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
  0%|          | 2/469 [00:00<00:50,  9.21it/s]

100%|██████████| 469/469 [00:23<00:00, 19.93it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 131.3000
	Loss_MB/train_phase/train_stream/Task000 = 4.7437
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0002
	Time_Epoch/train_phase/train_stream/Task000 = 23.5371
100%|██████████| 469/469 [00:23<00:00, 19.82it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 131.1000
	Loss_MB/train_phase/train_stream/Task000 = 2.6891
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0003
	Time_Epoch/train_phase/train_stream/Task000 = 23.6584
100%|██████████| 469/469 [00:23<00:00, 20.20it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 135.4000
	Loss_MB/train_phase/train_stream/Task000 = 2.2801
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 23.2214
100%|██████████| 469/469 [00:23<00:00, 19.60it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 120.8000
	Loss_MB/tr

  5%|▌         | 1/20 [03:33<1:07:34, 213.39s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 43.6205
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x169804dc0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.7541, 0.1786, 0.1538, 0.1292, 0.1022, 0.1716, 0.1376, 0.1653, 0.1622,
         0.1814, 0.1479, 0.1439, 0.1977, 0.1373, 0.1605, 0.1813, 0.1999, 0.1858,
         0.1335, 0.1845],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 52.1663
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x285896730>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.7541, 0.1786, 0.1538, 0.1292, 0.1022, 0.1716, 0.1376, 0.1653, 0.1622,
         0.1814, 0.1479, 0.1439, 0.1977, 0.1373, 0.1605, 0.1813, 0.1999, 0.1858,
         0.1335, 0.1845],
        [0.7275, 0.7865, 0.1805, 0.1723, 0.1329, 0.1097, 0.1838, 0.1258, 0.1397,
         0.2162, 0.2048, 0.1199, 0.2217, 0.1474, 0.2334, 0.1607, 0.2035, 0.1466,
         0.1542, 0.1991],
        [0.6910, 0.7805, 0.8073, 0.1284, 0.1131, 0.1402, 0.1417, 0.1555, 0.1288,
         0.1479, 0.1806, 0.1144, 0.1978, 0.1346, 0.2586, 0.0936, 0.1797, 0.1846,
         0.1829, 0.1399],
        [0.6383, 0.7570, 0.7926, 0.8130, 0.0891, 0.0926, 0.1659, 0.1928, 0.0991,
         0.1264, 0.1291, 0.1107, 0.2187, 0.1017, 0.22

 35%|███▌      | 7/20 [25:38<50:00, 230.83s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.4937
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x285e88e50>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.7541, 0.1786, 0.1538, 0.1292, 0.1022, 0.1716, 0.1376, 0.1653, 0.1622,
         0.1814, 0.1479, 0.1439, 0.1977, 0.1373, 0.1605, 0.1813, 0.1999, 0.1858,
         0.1335, 0.1845],
        [0.7275, 0.7865, 0.1805, 0.1723, 0.1329, 0.1097, 0.1838, 0.1258, 0.1397,
         0.2162, 0.2048, 0.1199, 0.2217, 0.1474, 0.2334, 0.1607, 0.2035, 0.1466,
         0.1542, 0.1991],
        [0.6910, 0.7805, 0.8073, 0.1284, 0.1131, 0.1402, 0.1417, 0.1555, 0.1288,
         0.1479, 0.1806, 0.1144, 0.1978, 0.1346, 0.2586, 0.0936, 0.1797, 0.1846,
         0.1829, 0.1399],
        [0.6383, 0.7570, 0.7926, 0.8130, 0.0891, 0.0926, 0.1659, 0.1928, 0.0991,
         0.1264, 0.1291, 0.1107, 0.2187, 0.1017, 0.22

 40%|████      | 8/20 [28:58<44:13, 221.15s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 52.3746
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x286ca1bb0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.7541, 0.1786, 0.1538, 0.1292, 0.1022, 0.1716, 0.1376, 0.1653, 0.1622,
         0.1814, 0.1479, 0.1439, 0.1977, 0.1373, 0.1605, 0.1813, 0.1999, 0.1858,
         0.1335, 0.1845],
        [0.7275, 0.7865, 0.1805, 0.1723, 0.1329, 0.1097, 0.1838, 0.1258, 0.1397,
         0.2162, 0.2048, 0.1199, 0.2217, 0.1474, 0.2334, 0.1607, 0.2035, 0.1466,
         0.1542, 0.1991],
        [0.6910, 0.7805, 0.8073, 0.1284, 0.1131, 0.1402, 0.1417, 0.1555, 0.1288,
         0.1479, 0.1806, 0.1144, 0.1978, 0.1346, 0.2586, 0.0936, 0.1797, 0.1846,
         0.1829, 0.1399],
        [0.6383, 0.7570, 0.7926, 0.8130, 0.0891, 0.0926, 0.1659, 0.1928, 0.0991,
         0.1264, 0.1291, 0.1107, 0.2187, 0.1017, 0.22

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
all_loss_components = strategy.get_store_loss()

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "all_loss_components": all_loss_components,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))